# Libraries


In [ ]:
!pip install imbalanced-learn
!pip install catboost
!pip install xgboost
!pip install tabpfn

In [ ]:
# --- Libraries ---

import logging
import sys
import os
from contextlib import contextmanager
import warnings
import time
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.stats import multivariate_normal, multivariate_t
from scipy.linalg import block_diag

from imblearn.over_sampling import SMOTE # Resampling method to cancel out imbalance - did not work well
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import roc_curve, balanced_accuracy_score, roc_auc_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.exceptions import ConvergenceWarning

from xgboost import XGBClassifier, callback

from catboost import CatBoostClassifier, Pool

from tabpfn import TabPFNClassifier

# Methods Selected vs. Tunning Situation

| Method | Tuning | Fairness | Settings (On/Off) |
|--------|--------|----------|-------------------|
| Linear Sparse LR | GridCV on C | Fair | Imbalance: Off; Device: CPU (n_jobs=1); Early Stop: NA ; Aug: Off |
| Augmented Sparse LR | GridCV on C post-aug | Fair | Imbalance: Off; Device: CPU (n_jobs=1); Early Stop: NA; Aug: On |
| Random Forest | GridCV on n_est/depth | Fair | Imbalance: Off; Device: CPU (n_jobs=1); Early Stop: NA; Aug: Off |
| XGBoost | GridCV on n_est/depth | Fair | Imbalance: Off; Device: CPU (hist, n_jobs=1); Early Stop: Off; Aug: Off |
| CatBoost | GridCV on iter/depth | Fair | Imbalance: Off; Device: CPU (default); Early Stop: Off; Aug: Off |
| TabPFN (No Tune) | None; fixed | Somewhat Fair | Imbalance: Off; Device: CPU; Early Stop: Off; Aug: Off; Suppress Output: On |
| XGBoost Early Stop | Early stop on n_est | Somewhat unfair | Imbalance: Off; Device: CPU (hist, n_jobs=1); Early Stop: On (50 rounds); Aug: Off |
| CatBoost Early Stop | Early stop on iter | Somewhat unfair | Imbalance: Off; Device: CPU (default); Early Stop: On (50 rounds); Aug: Off |
#| TabPFN (Tune) | GridCV on n_est | Fair | Imbalance: On (undersamp if imb); Device: CPU; Early Stop: Off; Aug: Off; Suppress Output: On |



# Model Runners

In [ ]:
import numpy as np

def augment_features(X):
    """Augment function: add pairwise feature interactions efficiently."""
    n, d = X.shape
    # Compute all pairwise interactions using matrix operations
    idx = np.triu_indices(d, k=0)  # Get indices for upper triangle (including diagonal)
    xx = X[:, idx[0]] * X[:, idx[1]]  # Element-wise multiplication for pairwise terms
    return np.hstack((X, xx))

In [ ]:
# --- Model Runners  ---

# Models are defined to ensure fairness in simulation comparison based on the following:

# 1. Hyperparameter Tuning: All models use GridSearchCV with a separate validation set (20% of training data,
#    stratified split, performed in the simulation loop) to tune key parameters (C for logistic regression,
#    n_estimators/max_depth for Random Forest/XGBoost/CatBoost, nothing for TabPFN since it claims no tunning
#    required), removing tuning bias and preventing overfitting to training data.

# 2. Class Imbalance: No imbalance handling; models train on original unbalanced data.

# 3. Model Complexity: Tree-based models use increased base complexity (n_estimators=100, max_depth=5 or 6)
#    as starting points for tuning to align with typical defaults.

# 4. Computational Resources: TabPFN set to device='cpu' to standardize hardware usage, ensuring fair runtime
#    comparisons.

# 5. Metrics: Retained original metrics (balanced accuracy, ROC AUC, MSE, FPR, TPR) for consistency with the
#    simulation function.

# Linear Sparse Logistic Regression
def run_linear_sparse_logistic_regression(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val):
    # Define model and tuning grid
    lr = LogisticRegression(
        penalty='l1',
        solver='saga',
        max_iter=2000,
        n_jobs=1,
        #class_weight='balanced'  # Handle class imbalance
    )
    param_grid = {'C': [0.001, 0.01, 0.1, 1.0, 10, 100]}  # Tune regularization strength
    grid_search = GridSearchCV(
        lr,
        param_grid,
        cv=[(np.arange(len(X_train_sub)), np.arange(len(X_train_sub), len(X_train_sub) + len(X_val)))],  # Train/val split
        scoring='roc_auc',
        n_jobs=1
    )
    grid_search.fit(np.vstack((X_train_sub, X_val)), np.hstack((y_train_sub, y_val)))
    best_model = grid_search.best_estimator_

    # Train best model on full training data
    best_model.fit(X_train, y_train)

    # Predict and compute metrics
    pred = best_model.predict(X_test)
    proba = best_model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, proba)
    return (
        balanced_accuracy_score(y_test, pred),
        roc_auc_score(y_test, proba),
        mean_squared_error(y_test, pred),
        fpr,
        tpr
    )

# Sparse Logistic Regression on Augmented Feature Space
def run_sparse_logistic_regression(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val):
    # Apply augmentation first
    Aug_X_train = augment_features(X_train)
    Aug_X_test = augment_features(X_test)
    Aug_X_train_sub = augment_features(X_train_sub)
    Aug_X_val = augment_features(X_val)

    # Define model and tuning grid
    lr = LogisticRegression(
        penalty='l1',
        solver='saga',
        max_iter=2000,
        n_jobs=1,
        #class_weight='balanced'  # Handle class imbalance
    )
    param_grid = {'C': [0.001, 0.01, 0.1, 1.0, 10, 100]}  # Tune regularization strength
    grid_search = GridSearchCV(
        lr,
        param_grid,
        cv=[(np.arange(len(Aug_X_train_sub)), np.arange(len(Aug_X_train_sub), len(Aug_X_train_sub) + len(Aug_X_val)))],  # Train/val split
        scoring='roc_auc',
        n_jobs=1
    )
    grid_search.fit(np.vstack((Aug_X_train_sub, Aug_X_val)), np.hstack((y_train_sub, y_val)))
    best_model = grid_search.best_estimator_

    # Train best model on full augmented training data
    best_model.fit(Aug_X_train, y_train)

    # Predict and compute metrics
    pred = best_model.predict(Aug_X_test)
    proba = best_model.predict_proba(Aug_X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, proba)
    return (
        balanced_accuracy_score(y_test, pred),
        roc_auc_score(y_test, proba),
        mean_squared_error(y_test, pred),
        fpr,
        tpr
    )

# Random Forest
def run_random_forest(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val):
    # Define model and tuning grid
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=5,
        n_jobs=1,  # Increased base complexity
        #class_weight='balanced'  # Handle class imbalance
    )
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, None]
    }  # Tune number of trees and depth
    grid_search = GridSearchCV(
        rf,
        param_grid,
        cv=[(np.arange(len(X_train_sub)), np.arange(len(X_train_sub), len(X_train_sub) + len(X_val)))],  # Train/val split
        scoring='roc_auc',
        n_jobs=1
    )
    grid_search.fit(np.vstack((X_train_sub, X_val)), np.hstack((y_train_sub, y_val)))
    best_model = grid_search.best_estimator_

    # Train best model on full training data
    best_model.fit(X_train, y_train)

    # Predict and compute metrics
    pred = best_model.predict(X_test)
    proba = best_model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, proba)
    return (
        balanced_accuracy_score(y_test, pred),
        roc_auc_score(y_test, proba),
        mean_squared_error(y_test, pred),
        fpr,
        tpr
    )

# XGBoost
def run_xgboost(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val):
    # Compute scale_pos_weight for imbalance
    #scale_pos_weight = sum(y_train == 0) / sum(y_train == 1) if sum(y_train == 1) > 0 else 1.0

    # Define model and tuning grid
    xgb = XGBClassifier(
        n_estimators=100,  # Increased base complexity
        max_depth=6,       # Default-like depth
        learning_rate=0.1,
        eval_metric='logloss',
        tree_method='hist',
        n_jobs=1
        #scale_pos_weight=scale_pos_weight,  # Handle class imbalance
    )
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 6]
    }  # Tune number of trees and depth
    grid_search = GridSearchCV(
        xgb,
        param_grid,
        cv=[(np.arange(len(X_train_sub)), np.arange(len(X_train_sub), len(X_train_sub) + len(X_val)))],  # Train/val split
        scoring='roc_auc',
        n_jobs=1
    )
    grid_search.fit(np.vstack((X_train_sub, X_val)), np.hstack((y_train_sub, y_val)))
    best_model = grid_search.best_estimator_

    # Train best model on full training data
    best_model.fit(X_train, y_train)

    # Predict and compute metrics
    pred = best_model.predict(X_test)
    proba = best_model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, proba)
    return (
        balanced_accuracy_score(y_test, pred),
        roc_auc_score(y_test, proba),
        mean_squared_error(y_test, pred),
        fpr,
        tpr
    )

# CatBoost
def run_catboost(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val):
    # Define model and tuning grid
    cb = CatBoostClassifier(
        iterations=100,  # Increased base complexity
        depth=6,         # Default-like depth
        learning_rate=0.1,
        verbose=0
        #auto_class_weights='Balanced',  # Handle class imbalance
    )
    param_grid = {
        'iterations': [50, 100, 200],
        'depth': [3, 5, 6]
    }  # Tune number of iterations and depth
    grid_search = GridSearchCV(
        cb,
        param_grid,
        cv=[(np.arange(len(X_train_sub)), np.arange(len(X_train_sub), len(X_train_sub) + len(X_val)))],  # Train/val split
        scoring='roc_auc',
        n_jobs=1
    )
    grid_search.fit(np.vstack((X_train_sub, X_val)), np.hstack((y_train_sub, y_val)))
    best_model = grid_search.best_estimator_

    # Train best model on full training data
    best_model.fit(X_train, y_train)

    # Predict and compute metrics
    pred = best_model.predict(X_test)
    proba = best_model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, proba)
    return (
        balanced_accuracy_score(y_test, pred),
        roc_auc_score(y_test, proba),
        mean_squared_error(y_test, pred),
        fpr,
        tpr
    )

# TabPFN (CPU-only)
def run_tabpfn(X_train, X_test, y_train, y_test, rs):

    # Conditionally subsample to balance classes for training
    #class_0_count = sum(y_train == 0)
    #class_1_count = sum(y_train == 1)
    #class_ratio = class_0_count / class_1_count if class_1_count > 0 else float('inf')
    #if class_ratio != 1:
    #    rus = RandomUnderSampler(random_state=rs)
    #    X_train_res, y_train_res = rus.fit_resample(X_train, y_train)
    #else:
    #    X_train_res, y_train_res = X_train, y_train

    X_train_res, y_train_res = X_train, y_train

    # Define model with no tuning (paper's default) "less tahn 1000 no hyperparameter tunning required"
    with suppress_output():
        tabpfn = TabPFNClassifier(
            n_estimators=1,  # Single model for instant inference
            device='cpu',  # Match other models
            inference_precision='auto',
            n_jobs=1,  # Minimal repeats
            ignore_pretraining_limits=True
        )
    # Single fit on resampled data
    with suppress_output():
        tabpfn.fit(X_train_res, y_train_res)

    # Predict and compute metrics
    pred = tabpfn.predict(X_test)
    proba = tabpfn.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, proba)
    return (
        balanced_accuracy_score(y_test, pred),
        roc_auc_score(y_test, proba),
        mean_squared_error(y_test, pred),
        fpr,
        tpr
    )

# TabPFN with hyperparameter tunning (CPU-only)
def run_tabpfn_large_sample(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val, rs):

    # Conditionally subsample to balance classes for training
    #class_0_count = sum(y_train == 0)
    #class_1_count = sum(y_train == 1)
    #class_ratio = class_0_count / class_1_count if class_1_count > 0 else float('inf')
    #if class_ratio != 1:
    #    rus = RandomUnderSampler(random_state=rs)
    #    X_train_res, y_train_res = rus.fit_resample(X_train, y_train)
    #    X_train_sub_res, y_train_sub_res = rus.fit_resample(X_train_sub, y_train_sub)
    #else:
    #    X_train_res, y_train_res = X_train, y_train
    #    X_train_sub_res, y_train_sub_res = X_train_sub, y_train_sub
    X_train_res, y_train_res = X_train, y_train
    X_train_sub_res, y_train_sub_res = X_train_sub, y_train_sub
    # Define model and tuning grid
    with suppress_output():  # Suppress output during model initialization
        tabpfn = TabPFNClassifier(
            n_estimators=4,  # Starting point
            device='cpu',    # CPU-only for hardware fairness
            inference_precision='auto',
            n_jobs=1,
            ignore_pretraining_limits=True
        )
    param_grid = {
        'n_estimators': [4, 8, 16]  # Tune number of ensembles (limited range due to computational cost)
    }
    grid_search = GridSearchCV(
        tabpfn,
        param_grid,
        cv=[(np.arange(len(X_train_sub_res)), np.arange(len(X_train_sub_res), len(X_train_sub_res) + len(X_val)))],  # Train/val split
        scoring='roc_auc',
        n_jobs=1
    )
    with suppress_output():  # Suppress output during grid search
        grid_search.fit(np.vstack((X_train_sub_res, X_val)), np.hstack((y_train_sub_res, y_val)))
    best_model = grid_search.best_estimator_

    # Train best model on full training data
    with suppress_output():  # Suppress output during final fit
        best_model.fit(X_train_res, y_train_res)

    # Predict and compute metrics
    pred = best_model.predict(X_test)
    proba = best_model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, proba)
    return (
        balanced_accuracy_score(y_test, pred),
        roc_auc_score(y_test, proba),
        mean_squared_error(y_test, pred),
        fpr,
        tpr
    )

# XGBoost with Early Stopping
def run_xgboost_early_stopping(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val):
    # Compute scale_pos_weight for imbalance
    #scale_pos_weight = sum(y_train == 0) / sum(y_train == 1) if sum(y_train == 1) > 0 else 1.0

    # Define model with early stopping parameters
    xgb = XGBClassifier(
        n_estimators=1000,  # Large number for early stopping
        max_depth=6,
        learning_rate=0.1,
        eval_metric='logloss',
        tree_method='hist',
        n_jobs=1,
        #scale_pos_weight=scale_pos_weight,
        early_stopping_rounds=50  # early_stopping_rounds
    )

    # Train with early stopping on validation set
    xgb.fit(
        X_train_sub,
        y_train_sub,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # Train final model on full training data using best iteration
    best_n_estimators = xgb.best_iteration if hasattr(xgb, 'best_iteration') and xgb.best_iteration else 100
    xgb_final = XGBClassifier(
        n_estimators=best_n_estimators,
        max_depth=6,
        learning_rate=0.1,
        eval_metric='logloss',
        tree_method='hist',
        n_jobs=1,
        #scale_pos_weight=scale_pos_weight
    )
    xgb_final.fit(X_train, y_train)

    # Predict and compute metrics
    pred = xgb_final.predict(X_test)
    proba = xgb_final.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, proba)
    return (
        balanced_accuracy_score(y_test, pred),
        roc_auc_score(y_test, proba),
        mean_squared_error(y_test, pred),
        fpr,
        tpr
    )

# CatBoost with Early Stopping
def run_catboost_early_stopping(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val):
    # Define model with early stopping
    cb = CatBoostClassifier(
        iterations=1000,  # Large number for early stopping
        depth=6,
        learning_rate=0.1,
        verbose=0,
        #auto_class_weights='Balanced',
        early_stopping_rounds=50  # For CatBoost, this can stay in constructor
    )

    # Train with early stopping on validation set
    cb.fit(
        X_train_sub,
        y_train_sub,
        eval_set=(X_val, y_val),
        verbose=False
    )

    # Train final model on full training data using best iteration
    best_iterations = cb.best_iteration_ if hasattr(cb, 'best_iteration_') and cb.best_iteration_ else 100
    cb_final = CatBoostClassifier(
        iterations=best_iterations,
        depth=6,
        learning_rate=0.1,
        verbose=0,
        #auto_class_weights='Balanced'
    )
    cb_final.fit(X_train, y_train)

    # Predict and compute metrics
    pred = cb_final.predict(X_test)
    proba = cb_final.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, proba)
    return (
        balanced_accuracy_score(y_test, pred),
        roc_auc_score(y_test, proba),
        mean_squared_error(y_test, pred),
        fpr,
        tpr
    )


# Simulation Function

In [ ]:
# --- Simulation Function for inputs of different settings ---

def simulation(n, mu, sigma1, sigma2, df=5, dist="Normal", iter=100, large_sample=0):
    """
    Run simulation for Normal or Elliptical distributions, storing MSE, ROC-AUC, runtimes, FPR, and TPR.

    Parameters:
    - n: Dictionary with keys 'n1_tr', 'n2_tr', 'n1_te', 'n2_te' for sample sizes.
    - mu: Dictionary with keys 'mu1', 'mu2' for class means.
    - sigma1, sigma2: Covariance matrices for both distributions.
    - df: Degrees of freedom for t-distribution (default=5).
    - dist: Distribution type, "Normal" or "Elliptical" (default="Normal").
    - iter: Number of iterations (default=100).

    Returns:
    - Dictionary with MSE, ROC-AUC, runtimes, FPR, and TPR for each model.
    """
    n1_tr, n2_tr = n['n1_tr'], n['n2_tr']
    n1_te, n2_te = n['n1_te'], n['n2_te']
    mu1, mu2 = mu['mu1'], mu['mu2']

    errors = {
        'L-SLR_MSE': [], 'L-SLR_AUC': [], 'L-SLR_Time': [], 'L-SLR_FPR': [], 'L-SLR_TPR': [],
        'SLR_MSE': [], 'SLR_AUC': [], 'SLR_Time': [], 'SLR_FPR': [], 'SLR_TPR': [],
        'RandomForest_MSE': [], 'RandomForest_AUC': [], 'RandomForest_Time': [], 'RandomForest_FPR': [], 'RandomForest_TPR': [],
        'XGBoost_MSE': [], 'XGBoost_AUC': [], 'XGBoost_Time': [], 'XGBoost_FPR': [], 'XGBoost_TPR': [],
        'CatBoost_MSE': [], 'CatBoost_AUC': [], 'CatBoost_Time': [], 'CatBoost_FPR': [], 'CatBoost_TPR': [],
        'TabPFN_MSE': [], 'TabPFN_AUC': [], 'TabPFN_Time': [], 'TabPFN_FPR': [], 'TabPFN_TPR': [],
        'y_test': None  # Store y_test from last iteration (optional, kept for compatibility)
    }

    for i in tqdm(range(iter), desc=f"Sim {dist} Iterations"):
        global current_iteration  # For warning handler
        current_iteration = i
        warnings.showwarning = capture_warnings

        if dist == "Normal":
            # Generate training and test data for Normal distribution
            Tr_x1 = multivariate_normal.rvs(mean=mu1, cov=sigma1, size=n1_tr)
            Tr_x2 = multivariate_normal.rvs(mean=mu2, cov=sigma2, size=n2_tr)
            Te_x1 = multivariate_normal.rvs(mean=mu1, cov=sigma1, size=n1_te)
            Te_x2 = multivariate_normal.rvs(mean=mu2, cov=sigma2, size=n2_te)
        elif dist == "Elliptical":
            # Compute scale matrices from covariance matrices
            if df <= 2:
                raise ValueError("Degrees of freedom must be > 2 for valid variance")
            scale1 = sigma1 * (df - 2) / df
            scale2 = sigma2 * (df - 2) / df
            # Generate training and test data for Elliptical (t-distribution)
            Tr_x1 = multivariate_t.rvs(loc=mu1, shape=scale1, df=df, size=n1_tr)
            Tr_x2 = multivariate_t.rvs(loc=mu2, shape=scale2, df=df, size=n2_tr)
            Te_x1 = multivariate_t.rvs(loc=mu1, shape=scale1, df=df, size=n1_te)
            Te_x2 = multivariate_t.rvs(loc=mu2, shape=scale2, df=df, size=n2_te)
        else:
            raise ValueError("dist must be 'Normal' or 'Elliptical'")

        X_train = np.vstack((Tr_x1, Tr_x2))
        y_train = np.array([0] * n1_tr + [1] * n2_tr)
        X_test = np.vstack((Te_x1, Te_x2))
        y_test = np.array([0] * n1_te + [1] * n2_te)

        # Split training data into train_sub and validation (80/20, stratified)
        X_train_sub, X_val, y_train_sub, y_val = train_test_split(
            X_train, y_train, test_size=0.2, stratify=y_train, random_state=i
        )

        # Run models and store MSE, ROC-AUC, runtimes, FPR, and TPR
        start_time = time.time()
        logreg_result = run_linear_sparse_logistic_regression(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val)
        errors['L-SLR_Time'].append(time.time() - start_time)
        errors['L-SLR_MSE'].append(logreg_result[2])
        errors['L-SLR_AUC'].append(logreg_result[1])
        errors['L-SLR_FPR'].append(logreg_result[3])
        errors['L-SLR_TPR'].append(logreg_result[4])

        start_time = time.time()
        auglogreg_result = run_sparse_logistic_regression(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val)
        errors['SLR_Time'].append(time.time() - start_time)
        errors['SLR_MSE'].append(auglogreg_result[2])
        errors['SLR_AUC'].append(auglogreg_result[1])
        errors['SLR_FPR'].append(auglogreg_result[3])
        errors['SLR_TPR'].append(auglogreg_result[4])

        start_time = time.time()
        rf_result = run_random_forest(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val)
        errors['RandomForest_Time'].append(time.time() - start_time)
        errors['RandomForest_MSE'].append(rf_result[2])
        errors['RandomForest_AUC'].append(rf_result[1])
        errors['RandomForest_FPR'].append(rf_result[3])
        errors['RandomForest_TPR'].append(rf_result[4])

        start_time = time.time()
        xgb_result = run_xgboost_early_stopping(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val)
        errors['XGBoost_Time'].append(time.time() - start_time)
        errors['XGBoost_MSE'].append(xgb_result[2])
        errors['XGBoost_AUC'].append(xgb_result[1])
        errors['XGBoost_FPR'].append(xgb_result[3])
        errors['XGBoost_TPR'].append(xgb_result[4])

        start_time = time.time()
        cb_result = run_catboost_early_stopping(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val)
        errors['CatBoost_Time'].append(time.time() - start_time)
        errors['CatBoost_MSE'].append(cb_result[2])
        errors['CatBoost_AUC'].append(cb_result[1])
        errors['CatBoost_FPR'].append(cb_result[3])
        errors['CatBoost_TPR'].append(cb_result[4])

        if large_sample==0:
          start_time = time.time()
          tabpfn_result = run_tabpfn(X_train, X_test, y_train, y_test, i)
          errors['TabPFN_Time'].append(time.time() - start_time)
          errors['TabPFN_MSE'].append(tabpfn_result[2])
          errors['TabPFN_AUC'].append(tabpfn_result[1])
          errors['TabPFN_FPR'].append(tabpfn_result[3])
          errors['TabPFN_TPR'].append(tabpfn_result[4])
        else:
          start_time = time.time()
          tabpfn_result = run_tabpfn_large_sample(X_train, X_test, y_train, y_test, X_train_sub, y_train_sub, X_val, y_val, i)
          errors['TabPFN_Time'].append(time.time() - start_time)
          errors['TabPFN_MSE'].append(tabpfn_result[2])
          errors['TabPFN_AUC'].append(tabpfn_result[1])
          errors['TabPFN_FPR'].append(tabpfn_result[3])
          errors['TabPFN_TPR'].append(tabpfn_result[4])
        # Store y_test for the last iteration
        if i == iter - 1:
            errors['y_test_last'] = y_test

    return errors

# Plot Fucntion

In [ ]:
#--- Plot Function to compare Runtime, MSE, and AUC ROC curve

def plot_results(errors, title):
    """
    Create Plotly box plots for runtimes and MSE, and average ROC curve plot with mean AUROC, in a 1x3 subplot layout.

    Parameters:
    - errors: Dictionary with MSE, AUC, runtimes, FPR, and TPR for each model.
    - title: Plot title for the setting.
    """
    # Extract model names (including TabPFN)
    models = ['TabPFN', 'SLR', 'L-SLR', 'RandomForest', 'XGBoost', 'CatBoost']

    # Create a 1x3 subplot layout
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=(
            "Runtime per Iteration",
            "Classification Error",
            "Average ROC Curve + Mean AUROC"
        ),
        specs=[[{"type": "box"}, {"type": "box"}, {"type": "scatter"}]],
        horizontal_spacing=0.1
    )

    # Box Plot for Runtimes (Column 1)
    for model in models:
        fig.add_trace(
            go.Box(
                y=errors[f'{model}_Time'],
                name=model,
                showlegend=False,
                boxpoints=False
            ),
            row=1, col=1
        )

    # Box Plot for MSE (Column 2)
    for model in models:
        fig.add_trace(
            go.Box(
                y=errors[f'{model}_MSE'],
                name=model,
                showlegend=False,
                boxpoints=False
            ),
            row=1, col=2
        )

    # Average ROC Curve Plot (Column 3)
    base_fpr = np.linspace(0, 1, 100)  # Fixed FPR grid for interpolation
    for model in models:
        # Interpolate TPR for each iteration to the base_fpr grid
        tpr_mean = []
        for fpr, tpr in zip(errors[f'{model}_FPR'], errors[f'{model}_TPR']):
            # Interpolate TPR at base_fpr points
            tpr_interp = np.interp(base_fpr, fpr, tpr)
            tpr_mean.append(tpr_interp)
        # Compute mean TPR across iterations
        tpr_mean = np.mean(tpr_mean, axis=0)
        mean_auc = np.mean(errors[f'{model}_AUC'])
        fig.add_trace(
            go.Scatter(
                x=base_fpr,
                y=tpr_mean,
                mode='lines',
                name=f"{model} (mean AUROC={mean_auc:.3f})",
                line=dict(width=2)
            ),
            row=1, col=3
        )
    # Add diagonal line (random classifier) to ROC plot
    fig.add_trace(
        go.Scatter(
            x=[0, 1], y=[0, 1],
            mode='lines',
            line=dict(dash='dash', color='black', width=1),
            showlegend=False
        ),
        row=1, col=3
    )

    # Update layout for each subplot
    fig.update_xaxes(title_text="Model", row=1, col=1)
    fig.update_yaxes(title_text="Seconds per Iteration", row=1, col=1)
    fig.update_xaxes(title_text="Model", row=1, col=2)
    fig.update_yaxes(title_text="Mean Squared Error", row=1, col=2)
    fig.update_xaxes(title_text="False Positive Rate (FPR)", row=1, col=3)
    fig.update_yaxes(title_text="True Positive Rate (TPR)", row=1, col=3)

    # Update overall layout
    fig.update_layout(
        title_text=title,
        showlegend=True,  # Legend only for ROC plot
        plot_bgcolor='white',
        paper_bgcolor='white',
        width=1200,
        height=400,
        grid=dict(rows=1, columns=3)
    )

    # Update axes to include gridlines
    fig.update_xaxes(gridcolor='lightgray', gridwidth=1, zerolinecolor='lightgray')
    fig.update_yaxes(gridcolor='lightgray', gridwidth=1, zerolinecolor='lightgray')

    # Show the combined figure
    fig.show()

In [ ]:
# Suppress stdout/stderr during TabPFN execution
@contextmanager
def suppress_output():
    """Context manager to temporarily suppress stdout and stderr."""
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

# List to store warnings with iteration number
captured_warnings = []

# Custom warning handler to capture warnings with iteration
def capture_warnings(message, category, filename, lineno, file=None, line=None):
    captured_warnings.append({
        'iteration': current_iteration,
        'message': str(message),
        'category': category.__name__,
        'filename': filename,
        'lineno': lineno
    })

# Suppress ConvergenceWarning display
warnings.filterwarnings('ignore', category=ConvergenceWarning)

# Suppress TabPFN logging output
logging.getLogger('tabpfn').setLevel(logging.ERROR)

# Simulation Results



## Setting 1: Small Sample, Low-Dimensional Gaussian

| **Parameter**                | **Mathematical Description**                                                                                     |
|------------------------------|------------------------------------------------------------------------------------------------------------------|
| **Distribution**             | $\mathcal{N}(\mu, \Sigma)$                                                                                       |
| **Dimensionality**           | $p = 20$                                                                                                         |
| **Training Size**            | $n_{\text{train}} = 200$  ($n_1 = n_2 = 100$)                                                                    |
| **Test Size**                | $n_{\text{test}} = 1{,}000$  ($n_1 = n_2 = 500$)                                                                  |
| **Class 0 Mean**             | $\mu_1 = \mathbf{0}_{20} = \begin{bmatrix} 0 \\ \vdots \\ 0 \end{bmatrix}$                                       |
| **Class 1 Mean**             | $\mu_2 = \begin{bmatrix} 0.7 \\ \vdots \\ 0.7 \\ 0 \\ \vdots \\ 0 \end{bmatrix}_{20\times1}$ (first 10 = 0.7)   |
| **Class 0 Covariance**       | $\Sigma_1 = \mathbf{I}_{20}$  (identity matrix)                                                                 |
| **Class 1 Covariance**       | $\Sigma_2 = \text{diag}(\underbrace{1.3, \ldots, 1.3}_{10}, \underbrace{1, \ldots, 1}_{10})$                     |
| **Signal Features**          | First 10 dimensions carry separation signal                                                                      |
| **Noise Features**           | Last 10 dimensions are pure noise (mean 0, variance 1)                                                           |
| **Separation Strength**      | $\Delta\mu = 0.7$ on 10/20 features → moderate overlap                                                           |
| **Monte Carlo Repeats**      | $B = 100$                                                                                                         |
| **Validation**               | 80/20 stratified split inside training                                                                           |


### Why this setting is useful

| Goal | How Setting 1 helps |
|------|---------------------|
| **Small-data reality check** | Only 200 training points → forces every model to fight **over-fitting** |
| **Partial signal** | Only 10/20 features move → tests **feature-selection power** (L1, trees, TabPFN) |
| **Mild heteroscedasticity** | Class-1 has 30 % higher variance in signal dims → breaks homoscedastic assumptions of plain Logistic Regression |
| **Real-world proxy** | 50 % noise features + moderate shift = typical tabular dataset |


In [ ]:
# --- Simulation Settings ---

# Setting 1: Small Sample, Low-Dimensional Gaussian
n1 = {'n1_tr': 100, 'n2_tr': 100, 'n1_te': 500, 'n2_te': 500}
mu1 = {'mu1': np.zeros(20), 'mu2': np.array([0.7]*10 + [0]*(10))}
sigma1_1 = np.eye(20)
sigma1_2 = np.diag([1.3]*10 + [1]*(10))
errors1 = simulation(n1, mu1, sigma1_1, sigma1_2, dist="Normal", iter=30)

# Plot for each setting
plot_results(errors1, 'Setting 1: Small Sample, Low-Dimensional Gaussian')

# Print captured warnings
#for w in captured_warnings:
#    print(f"Iteration {w['iteration']}: {w['category']}: {w['message']} at {w['filename']}:{w['lineno']}")

Sim Normal Iterations: 100%|██████████| 30/30 [18:29<00:00, 37.00s/it]


## Setting 2: Nonlinear Interactions with Noise

| **Parameter**                | **Mathematical Description**                                                                                     |
|-------------------|------------------------------------------------------------------------------------------------------------------|
| **Distribution**             | $\mathcal{N}(\mu, \Sigma)$                                                                                       |
| **Dimensionality**           | $p = 20$                                                                                                         |
| **Training Size**            | $n_{\text{train}} = 200$  ($n_1 = n_2 = 100$)                                                                    |
| **Test Size**                | $n_{\text{test}} = 1{,}000$  ($n_1 = n_2 = 500$)                                                                  |
| **Class Means**              | $\mu_1 = \mu_2 = \mathbf{0}_{20}$ → **zero mean shift**                                                          |
| **Covariance – Class 0**     | $\Sigma_1 = \text{blockdiag}\left(0.6\mathbf{I}_{10} + 0.4\mathbf{1}\mathbf{1}^\top,\ \mathbf{I}_{10}\right)$     |
| **Covariance – Class 1**     | $\Sigma_2 = \big(\Sigma_1^{-1} + \mathbf{I}_{20}\big)^{-1}$                                                      |
| **Correlation Structure**    | Features 1–10: **ρ = 0.4** within block <br> Features 11–20: **ρ = 0**                                           |
| **Key Insight**              | **In this model, neither $\Sigma_1^{-1}$ nor $\Sigma_2^{-1}$ is sparse,<br>but $\Sigma_1^{-1} - \Sigma_2^{-1}$ is.** |
| **Signal Type**              | Pure **quadratic decision boundary** via covariance contrast                                                    |
| **Monte Carlo Repeats**      | $B = 100$                                                                                                         |

### Why this setting is useful

| Goal | How Setting 2 delivers |
|------|------------------------|
| **Sparse precision *difference*** | $\Sigma_1^{-1} - \Sigma_2^{-1} = \mathbf{I}$ → **exact sparse signal** for methods like sparse LDA or graphical models |
| **Linear models fail** | No mean shift + dense $\Sigma^{-1}$ → **logistic regression blind** |
| **Winner** | Can capture block interactions and variance shifts |
| **Theory** | Clean math: **one identity matrix** explains *all* separation |



In [ ]:
# Setting 2: Nonlinear Interactions with Noise
n2 = {'n1_tr': 100, 'n2_tr': 100, 'n1_te': 500, 'n2_te': 500}
block1 = np.full((10, 10), 0.4) + np.eye(10) * (1 - 0.4)
block2 = np.eye(10)
sigma2_1 = block_diag(block1, block2)
sigma2_2 = np.linalg.inv(np.linalg.inv(sigma2_1) + np.eye(20))
mu2 = {'mu1': np.zeros(20), 'mu2': np.zeros(20)}
errors2 = simulation(n2, mu2, sigma2_1, sigma2_2, dist="Normal", iter=30)

# Plot for each setting
plot_results(errors2, 'Setting 2: Nonlinear Interactions with Noise')

# Print captured warnings
#for w in captured_warnings:
#    print(f"Iteration {w['iteration']}: {w['category']}: {w['message']} at {w['filename']}:{w['lineno']}")

Sim Normal Iterations: 100%|██████████| 30/30 [19:01<00:00, 38.05s/it]


## Setting 3: Nonlinear + Sparse Mean Signal

| **Parameter**                | **Mathematical Description**                                                                                     |
|------------------------------|------------------------------------------------------------------------------------------------------------------|
| **Distribution**             | $\mathcal{N}(\mu, \Sigma)$                                                                                       |
| **Dimensionality**           | $p = 20$                                                                                                         |
| **Training Size**            | $n_{\text{train}} = 200$  ($n_1 = n_2 = 100$)                                                                    |
| **Test Size**                | $n_{\text{test}} = 1{,}000$  ($n_1 = n_2 = 500$)                                                                  |
| **Class 0 Mean**             | $\mu_1 = \mathbf{0}_{20}$                                                                                        |
| **Class 1 Mean**             | $\mu_2 = \begin{bmatrix} \mathbf{1}_5 \\ \mathbf{0}_{15} \end{bmatrix}$ → **only 5/20 features shift**         |
| **Mean Difference**          | $\Delta\mu = \mu_2 - \mu_1 \in \mathbb{R}^{20}$ has **exact sparsity 5**                                          |
| **Covariance**               | **Same as Setting 2**: <br>$\Sigma_1 = \text{blockdiag}(0.6\mathbf{I}_{10} + 0.4\mathbf{1}\mathbf{1}^\top, \mathbf{I}_{10})$ <br>$\Sigma_2 = (\Sigma_1^{-1} + \mathbf{I}_{20})^{-1}$ |
| **Precision Diff**           | $\Sigma_1^{-1} - \Sigma_2^{-1} = \mathbf{I}_{20}$ → **sparse in precision space**                                |
| **Signal Cocktail**          | **Two independent sources**: <br>1. **Sparse mean shift** (5 features) <br>2. **Dense covariance contrast** (block correlation + precision gap) |
| **Decision Boundary**        | **Highly nonlinear** (quadratic + linear terms)                                                                  |
| **Monte Carlo Repeats**      | $B = 100$                                                                                                         |

### Why this setting is useful

| Goal | How Setting 3 delivers |
|------|------------------------|
| **Dual-signal torture** | Forces models to detect **both** sparse mean AND interaction patterns |
| **Sparsity hunting** | Only 5/20 features have mean shift |
| **Nonlinear + linear mix** | Combines **quadratic boundary** (from Σ) with **sparse hyperplane** (from μ) |
| **Real-world killer** | Mimics genomics, fraud, or medical data: **few strong SNPs + correlated gene blocks** |
| **Theoretical gold** | $\Delta\mu$ sparse **and** $\Sigma_1^{-1} - \Sigma_2^{-1}$ sparse → **double sparsity** |




In [ ]:
# Setting 3: Nonlinear Interactions with Sparse Mean Difference
n3 = {'n1_tr': 100, 'n2_tr': 100, 'n1_te': 500, 'n2_te': 500}

# Use same covariance structure as Model 2
sigma3_1 = sigma2_1
sigma3_2 = sigma2_2

# Means: μ1 = 0 (same as Model 2), μ2 with sparse signal (sparse mean difference)
mu3 = {'mu1': np.zeros(20),
       'mu2': np.concatenate([np.ones(5), np.zeros(15)])}  # First 5 features have mean difference

errors3 = simulation(n3, mu3, sigma3_1, sigma3_2, dist="Normal", iter=30)

# Plot for Model 3
plot_results(errors3, 'Setting 3: Nonlinear Interactions with Sparse Mean Difference')

Sim Normal Iterations: 100%|██████████| 30/30 [19:12<00:00, 38.42s/it]


## Setting 4: Class Imbalance

| **Parameter**                | **Mathematical Description**                                                                                     |
|------------------------------|------------------------------------------------------------------------------------------------------------------|
| **Distribution**             | $\mathcal{N}(\mu, \Sigma)$                                                                                       |
| **Dimensionality**           | $p = 20$                                                                                                         |
| **Training Size**            | $n_{\text{train}} = 200$  ($n_1 = 160$, $n_2 = 40$) → **80/20 split**                                            |
| **Test Size**                | $n_{\text{test}} = 1{,}000$  ($n_1 = 800$, $n_2 = 200$) → **same π = 0.2**                                       |
| **Class Prior (minority)**   | $\pi_+ = 0.2$  (positive class = 20 % of data)                                                                   |
| **Class 0 Mean**             | $\mu_1 = \mathbf{0}_{20}$                                                                                        |
| **Class 1 Mean**             | $\mu_2 = [0.7, \ldots, 0.7, 0, \ldots, 0]^\top$  (first 10 features)                                            |
| **Covariance – Class 0**     | $\Sigma_1 = \mathbf{I}_{20}$                                                                                     |
| **Covariance – Class 1**     | $\Sigma_2 = \text{diag}(1.3 \times 10, 1 \times 10)$                                                             |
| **Signal Features**          | First 10 dims: mean + variance shift                                                                             |
| **Noise Features**           | Last 10 dims: pure noise                                                                                         |
| **Separation Strength**      | $\Delta\mu = 0.7$ on 10/20 → **moderate**, but **rare class** makes it hard                                      |
| **Monte Carlo Repeats**      | $B = 100$                                                                                                         |
| **Validation**               | 80/20 **stratified** split (preserves π = 0.2)                                                                   |

### Why this setting is useful

| Goal | How Setting 4 punishes weak models |
|------|-------------------------------------|
| **Imbalanced learning** | Only **40 positive training points**  |
| **No free lunch** | Accuracy = 80 % by doing nothing →  AUC** required |
| **Rare signal + noise** | Minority class has **stronger variance** → trees overfit, LR underfits |
| **Real-world proxy** | Fraud, disease, churn: **π ∈ [0.01, 0.2]** is the norm |
| **Fairness** | With `stratify`, validation leaks prevented  |




In [ ]:
# Setting 4: Class Imbalance
n4 = {'n1_tr': 160, 'n2_tr': 40, 'n1_te': 800, 'n2_te': 200}  # π=0.2 for minority
mu4 = {'mu1': np.zeros(20), 'mu2': np.array([0.7]*10 + [0]*(10))}
sigma4_1 = np.eye(20)
sigma4_2 = np.diag([1.3]*10 + [1]*(10))
errors4 = simulation(n4, mu4, sigma4_1, sigma4_2, dist="Normal", iter=30)

# Plot for each setting
plot_results(errors4, 'Setting 4: Class Imbalance')

# Print captured warnings
#for w in captured_warnings:
#    print(f"Iteration {w['iteration']}: {w['category']}: {w['message']} at {w['filename']}:{w['lineno']}")

Sim Normal Iterations: 100%|██████████| 30/30 [18:46<00:00, 37.54s/it]


## Setting 5: Heavy-Tailed Elliptical

| **Parameter**                | **Mathematical Description**                                                                                     |
|------------------------------|------------------------------------------------------------------------------------------------------------------|
| **Distribution**             | $\mathbf{X} \sim t_{p}(\nu, \mu, \boldsymbol{S})$  → **multivariate t-distribution**                     |
| **Degrees of Freedom**       | $\nu = 5$ → **heavy tails**, infinite 4th moment, **real outliers**                                             |
| **Dimensionality**           | $p = 20$                                                                                                         |
| **Training Size**            | $n_{\text{train}} = 200$  ($n_1 = n_2 = 100$)                                                                    |
| **Test Size**                | $n_{\text{test}} = 1{,}000$  ($n_1 = n_2 = 500$)                                                                  |
| **Class 0 Mean**             | $\mu_1 = \mathbf{0}_{20}$                                                                                        |
| **Class 1 Mean**             | $\mu_2 = [0.7 \times 10, 0 \times 10]^\top$                                                                      |
| **Scale Matrix – Class 0**   | $\boldsymbol{S}_1 = \mathbf{I}_{20}$                                                                        |
| **Scale Matrix – Class 1**   | $\boldsymbol{S}_2 = \text{diag}(1.3 \times 10, 1 \times 10)$                                                 |
| **True Covariance**          | $\Sigma_i = \frac{\nu}{\nu-2} \boldsymbol{S}_i$ → **valid variance** (ν>2)                                  |
| **Outlier Frequency**        | ~5–10 % extreme points per batch → **breaks normality assumptions**                                              |
| **Separation**               | Same mean/variance shift as Setting 1 → but now **corrupted by wild jumps**                                     |
| **Monte Carlo Repeats**      | $B = 100$                                                                                                         |

### Why this setting is useful

| Goal | How Setting 5 destroys weak models |
|------|-------------------------------------|
| **Break Gaussian assumptions** | t₅ has **no 4th moment** → sample covariance **unstable** |
| **Punish robust-less models** | Trees overfit outliers  |
| **Test tail modeling** | Only models with **implicit or explicit heavy-tail handling** survive |
| **Real data proxy** | Finance, sensor noise, biology → **t-distributions are everywhere** |
| **Ultimate stress test** | Same signal as Setting 1… but now **drowned in outliers** |




In [ ]:
# Setting 5: Heavy-Tailed Distributions
n5 = {'n1_tr': 100, 'n2_tr': 100, 'n1_te': 500, 'n2_te': 500}
df = 5
mu5 = {'mu1': np.zeros(20), 'mu2': np.array([0.7]*10 + [0]*(10))}
sigma5_1 = np.eye(20)
sigma5_2 = np.diag([1.3]*10 + [1]*(10))
errors5 = simulation(n5, mu5, sigma5_1, sigma5_2, df=df, dist="Elliptical", iter=30)

# Plot for each setting
plot_results(errors5, 'Setting 5: Heavy-Tailed Distributions')

# Print captured warnings
#for w in captured_warnings:
#    print(f"Iteration {w['iteration']}: {w['category']}: {w['message']} at {w['filename']}:{w['lineno']}")

Sim Elliptical Iterations: 100%|██████████| 30/30 [20:37<00:00, 41.25s/it]


### **Setting 6: Heavy-Tailed with Exponential Covariance**

| **Parameter**                | **Mathematical Description**                                                                                     |
|------------------------------|------------------------------------------------------------------------------------------------------------------|
| **Distribution**             | $\mathbf{X} \sim t_{p}(\nu, \mu, \boldsymbol{S})$  → **multivariate t-distribution**                     |
| **Marginal Distribution**    | $t_{\nu}$ with $\nu = 5$ (implied heavy tails) → **inherent outliers**                                          |
| **Dimensionality**           | $p = 20$                                                                                                         |
| **Training Size**            | $n_{\text{train}} = 200$  ($n_1 = n_2 = 100$)                                                                    |
| **Test Size**                | $n_{\text{test}} = 1{,}000$  ($n_1 = n_2 = 500$)                                                                  |
| **Class 0 Mean**             | $\mu_1 = \mathbf{0}_{20}$                                                                                        |
| **Class 1 Mean**             | $\mu_2 = [0.7 \times 10, 0 \times 10]^\top$                                                                      |
| **Covariance – Class 0**     | $\boldsymbol{\Sigma}_1 = \mathbf{I}_{20}$                                                                        |
| **Covariance – Class 1**     | $\boldsymbol{\Sigma}_2(i, j) = 0.6^{|i-j|}$ → **Toeplitz structure with exponential decay**                    |
| **Covariance Nature**        | **Non-diagonal**, introduces **feature dependence** that is **local along variable indices**                     |
| **Separation**               | Combination of **mean shift** and **complex covariance shift** in a **heavy-tailed** environment                |
| **Monte Carlo Repeats**      | $B = 30$                                                                                                         |

#### Why this setting is useful

| Goal | How Setting 6 destroys weak models |
|------|-------------------------------------|
| **Break i.i.d. feature assumptions** | Exponential decay creates **local correlation structure** → breaks naive feature independence |
| **Test covariance modeling** | Models must handle **non-identity covariance** in **highly non-Gaussian** data |
| **Combined complexity** | **Three challenges at once**: heavy tails, mean shift, and complex covariance shift |
| **Real data proxy** | Genetic data, time-series, spatial processes → where **proximity implies correlation** |
| **Feature selection stress test** | Correlated, heavy-tailed features make **stable feature selection difficult** |

These tables accurately reflect the parameters and purposes of your simulation settings 5 and 6.

In [ ]:
# Setting 6: Heavy-Tailed Distributions with Exponential Covariance Decay
n6 = {'n1_tr': 100, 'n2_tr': 100, 'n1_te': 500, 'n2_te': 500}
df = 5
mu6 = {'mu1': np.zeros(20), 'mu2': np.array([0.7]*10 + [0]*(10))}
sigma6_1 = np.eye(20)
sigma6_2 = np.zeros((20, 20)) # Σ2 has exponential off-diagonal decay: Σ2(i, j) = 0.6^|i−j|
for i in range(20):
    for j in range(20):
        sigma6_2[i, j] = 0.6 ** abs(i - j)

errors6 = simulation(n6, mu6, sigma6_1, sigma6_2, dist="Elliptical", iter=30)

# Plot for Model 6
plot_results(errors6, 'Setting 6: Heavy-Tailed Distributions with Exponential Covariance Decay')

# Print captured warnings
#for w in captured_warnings:
#    print(f"Iteration {w['iteration']}: {w['category']}: {w['message']} at {w['filename']}:{w['lineno']}")

Sim Elliptical Iterations: 100%|██████████| 30/30 [19:28<00:00, 38.94s/it]


Of course! Here is the formatted table for your Setting 7.

---

### **Setting 7: High-Dimensional Sparse Gaussian**

| **Parameter**                | **Mathematical Description**                                                                                     |
|------------------------------|------------------------------------------------------------------------------------------------------------------|
| **Distribution**             | $\mathbf{X} \sim \mathcal{N}_{p}(\mu, \boldsymbol{\Sigma})$  → **Multivariate Gaussian**                       |
| **Dimensionality**           | $p = 40$                                                                                                         |
| **Training Size**            | $n_{\text{train}} = 200$  ($n_1 = n_2 = 100$)                                                                    |
| **Test Size**                | $n_{\text{test}} = 1{,}000$  ($n_1 = n_2 = 500$)                                                                  |
| **Class 0 Mean**             | $\mu_1 = \mathbf{0}_{40}$                                                                                        |
| **Class 1 Mean**             | $\mu_2 = [0.7 \times 5, 0 \times 35]^\top$                                                                       |
| **Covariance – Class 0**     | $\boldsymbol{\Sigma}_1 = \mathbf{I}_{40}$                                                                        |
| **Covariance – Class 1**     | $\boldsymbol{\Sigma}_2 = \text{diag}(1.3 \times 5, 1 \times 35)$                                                 |
| **Sparsity**                 | **Only 5 out of 40 features (12.5%) are relevant** ($\delta = 0.7$, $\sigma^2_{\text{diff}} = 1.3$)              |
| **Data Regime**              | $p > n_{\text{train}}$ → Close to **High-dimensional regime**                                                             |
| **Separation**               | **Sparse mean shift** and **variance shift** concentrated in a small subset of features                          |
| **Monte Carlo Repeats**      | $B = 100$                                                                                                         |

#### Why this setting is useful

| Goal | How Setting 7 challenges models |
|------|-------------------------------------|
| **Test high-dimensional performance** | $p > n$ regime → **curse of dimensionality** becomes active |
| **Evaluate feature selection** | Models must **identify the 5 relevant features** among 40 to avoid overfitting |
| **Punish non-sparse models** | Methods without implicit or explicit **sparsity** will overfit to noise variables |
| **Real data proxy** | Genomics, text data, many modern ML tasks → **many features, few samples** |
| **Simplicity as a baseline** | Despite high dimension, data is Gaussian → tests if models can handle **dimension alone without other complexities** |



In [ ]:
# Setting 7: High-Dimensional Sparse Gaussian
n7 = {'n1_tr': 100, 'n2_tr': 100, 'n1_te': 500, 'n2_te': 500}
mu7 = {'mu1': np.zeros(40), 'mu2': np.array([0.7]*5 + [0]*(35))}
sigma7_1 = np.eye(40)
sigma7_2 = np.diag([1.3]*5 + [1]*(35))
errors7 = simulation(n7, mu7, sigma7_1, sigma7_2, dist="Normal", iter=30) # not enough compute power

# Plot for each setting
plot_results(errors7, 'Setting 7: High-Dimensional Sparse Gaussian')

# Print captured warnings
#for w in captured_warnings:
#    print(f"Iteration {w['iteration']}: {w['category']}: {w['message']} at {w['filename']}:{w['lineno']}")

Sim Normal Iterations: 100%|██████████| 30/30 [39:06<00:00, 78.23s/it]


Of course! Here is the formatted table for your Setting 8.

---

### **Setting 8: Ultra-High Dimensional Sparse Gaussian**

| **Parameter**                | **Mathematical Description**                                                                                     |
|------------------------------|------------------------------------------------------------------------------------------------------------------|
| **Distribution**             | $\mathbf{X} \sim \mathcal{N}_{p}(\mu, \boldsymbol{\Sigma})$  → **Multivariate Gaussian**                       |
| **Dimensionality**           | $p = 100$                                                                                                        |
| **Training Size**            | $n_{\text{train}} = 200$  ($n_1 = n_2 = 100$)                                                                    |
| **Test Size**                | $n_{\text{test}} = 1{,}000$  ($n_1 = n_2 = 500$)                                                                  |
| **Class 0 Mean**             | $\mu_1 = \mathbf{0}_{100}$                                                                                       |
| **Class 1 Mean**             | $\mu_2 = [0.7 \times 5, 0 \times 95]^\top$                                                                       |
| **Covariance – Class 0**     | $\boldsymbol{\Sigma}_1 = \mathbf{I}_{100}$                                                                       |
| **Covariance – Class 1**     | $\boldsymbol{\Sigma}_2 = \text{diag}(1.3 \times 5, 1 \times 95)$                                                 |
| **Sparsity**                 | **Only 5 out of 100 features (5%) are relevant** ($\delta = 0.7$, $\sigma^2_{\text{diff}} = 1.3$)               |
| **Data Regime**              | $p >> n_{\text{train}}$ → **Ultra-high-dimensional regime**                                                       |
| **Dimensionality Ratio**     | $p/n_{\text{train}} = 0.5$ → **Extreme challenge for non-sparse methods**                                        |
| **Separation**               | **Extremely sparse signal** in a **high-noise environment**                                                      |
| **Monte Carlo Repeats**      | $B = 100$                                                          |

#### Why this setting is useful

| Goal | How Setting 8 challenges models |
|------|-------------------------------------|
| **Stress test for scalability** | **p = 100** dimensions tests computational limits and algorithmic efficiency |
| **Evaluate extreme sparsity** | Models must find **needle in a haystack** - only 5% of features are relevant |
| **Break non-sparse methods** | Methods without strong **regularization/sparsity** will completely fail due to overfitting |
| **Real data proxy** | Modern bioinformatics (e.g., gene expression), neuroimaging, any **very wide data** |
| **Separate feature selectors** | Distinguishes models with **effective high-dim feature selection** from those that only work in low dimensions |



In [ ]:
# Setting 8: Ultra-High Dimensionality
n8 = {'n1_tr': 100, 'n2_tr': 100, 'n1_te': 500, 'n2_te': 500}
mu8 = {'mu1': np.zeros(100), 'mu2': np.array([0.7]*5 + [0]*(95))}
sigma8_1 = np.eye(100)
sigma8_2 = np.diag([1.3]*5 + [1]*(95))
errors8 = simulation(n8, mu8, sigma8_1, sigma8_2, dist="Normal", iter=10) # not enough compute power

# Plot for each setting
plot_results(errors8, 'Setting 8: Ultra-High Dimensionality')

# Print captured warnings
#for w in captured_warnings:
#    print(f"Iteration {w['iteration']}: {w['category']}: {w['message']} at {w['filename']}:{w['lineno']}")

Sim Normal Iterations: 100%|██████████| 10/10 [55:46<00:00, 334.66s/it]
